In [ ]:
import numpy as np
import kerrgeopy as kg
from math import gcd

In [ ]:
def q_of_E(E, a, L, Q):
    """
    给定 E, a, L, Q，返回 q(E) = upsilon_phi / upsilon_r
    """
    orbit = kg.StableOrbit.from_constants(a, E, L, Q)
    upsilon_r, upsilon_theta, upsilon_phi, gamma = orbit.mino_frequencies()
    return upsilon_phi / upsilon_r

def find_bracket_for_q_target(q_target, E_values, q_values):
    """
    在离散采样点 E_values, q_values 中，
    找到能把 q_target 夹在中间的 (E_low, q_low) 和 (E_high, q_high)。
    要求 q_low <= q_target <= q_high 或反过来。
    如果没能找到，则返回 None。
    """
    for i in range(len(E_values) - 1):
        q1, q2 = q_values[i], q_values[i+1]
        # 判断 q_target 是否被 [q1, q2] 包含（不要求绝对顺序，可正可反）
        if (q1 <= q_target <= q2) or (q2 <= q_target <= q1):
            return (E_values[i], q1, E_values[i+1], q2)
    return None

def bisection_for_q_target(q_target, E_low, E_high, a, L, Q, tol=1e-6, max_iter=100):
    """
    在 [E_low, E_high] 区间内做二分搜索，找使得 q(E) ∈ [q_target, q_target + tol] 的 E。
    注意这里的 tol 用于判断 q(E) 与 q_target 的接近度。
    max_iter 为迭代次数上限。
    """
    for _ in range(max_iter):
        E_mid = 0.5 * (E_low + E_high)
        q_mid = q_of_E(E_mid, a, L, Q)

        # 如果 q_mid 落在 [q_target, q_target+tol] 里面，认为“找到”了
        if q_target <= q_mid <= q_target + tol:
            return E_mid

        # 根据单调性做判断（假设 q(E) 随 E 单调递增，如果相反则需要调换判断逻辑）
        if q_mid < q_target:
            E_low = E_mid
        else:
            E_high = E_mid

    # 如果 max_iter 达到还没退出，就返回 E_mid 作为近似
    return 0.5 * (E_low + E_high)
def get_exponential_sampling(E_min, E_max, n_points):
    """
    返回从 E_min 到 E_max 的指数级（几何级数）采样，共 n_points 个点。
    确保返回的最大采样值不会超过 E_max。
    """
    if E_min <= 0:
        raise ValueError("E_min 必须是正数，否则无法做几何采样。")
    if E_min >= E_max:
        raise ValueError("E_min 必须严格小于 E_max。")
    if n_points < 2:
        raise ValueError("n_points 至少要大于 1，才能形成区间。")

    E_values = np.zeros(n_points, dtype=float)

    # 计算等比因子 ratio，使得 E_min * ratio^(n_points-1) = E_max
    ratio = (E_max / E_min) ** (1.0 / (n_points - 1))

    val = E_min
    E_values[0] = val

    # 依次累乘 ratio
    for i in range(1, n_points):
        val *= ratio
        # 若浮点误差导致 val 稍大于 E_max，做截断
        if val > E_max:
            val = E_max
        E_values[i] = val

    # 也可以再一次确保最后一个点是 E_max
    E_values[-1] = E_max

    return E_values

def main(a, L, Q, E_min, E_max, n=1000, tol=1e-6):
    """
    主函数：先在 [E_min, E_max] 做离散采样，然后对各个 q_target 做二分逼近。
    最终把结果打印出来。
    """
    # 1) 先做粗采样
    E_values = get_exponential_sampling(E_min, E_max, n)
    #E_values = np.linspace(E_min, E_max, n)
    q_values = np.zeros_like(E_values)

    for i, E in enumerate(E_values):
        q_values[i] = q_of_E(E, a, L, Q)

    # 注意：如果 q(E) 并不是随 E 单调递增，那么要先判断一下，如果是递减的，
    # 可能需要翻转 q_values 或者调换二分搜索中的判断条件。
    # 这里暂时假设它是单调递增

    # 2) 枚举所有 w, z, v
    #    根据需求：
    #    w = 1, 2, 3
    #    z = 1, 2, 3, 4, 5
    #    v in [0,z), 但只有当 z=1 时才有 v=0，否则 v=1,2,...,z-1
    results = []

    for w in [0, 1, 2, 3, 4, 5]:
        for z in [1, 2, 3, 4, 5]:
            if z == 1:
                # 特例：v=0
                v_list = [0]
            else:
                # 普通情况：1 <= v < z, 且 gcd(z, v)=1
                v_list = [v for v in range(1, z) if gcd(z, v) == 1]

            for v in v_list:
                q_target = 1 + w + v / z  # 目标 q

                # 寻找在粗采样中包住 q_target 的 bracket
                bracket = find_bracket_for_q_target(q_target, E_values, q_values)
                if bracket is None:
                    # 没有在采样中找到能夹住 q_target 的区间
                    # 说明 q_target 可能超出 [min(q), max(q)] 范围
                    continue

                E_low, q_low, E_high, q_high = bracket

                # 用二分搜索精细逼近
                E_target = bisection_for_q_target(q_target, E_low, E_high,
                                                  a, L, Q, tol=tol)
                # 再计算一下逼近后的 q 值
                q_approx = q_of_E(E_target, a, L, Q)

                # 判断是否成功落入 [q_target, q_target+tol] 区间
                if q_target <= q_approx <= q_target + tol:
                    results.append((z, w, v, E_target, q_approx))

    # 3) 打印或返回结果
    print("=== Results (z, w, v, E, q) ===")
    for (z, w, v, E_val, q_val) in results:
        print(f"z={z}, w={w}, v={v}, E={E_val:.12f}, q={q_val:.9f}")

    return results


In [ ]:
if __name__ == "__main__":
    # 这里给出示例参数，供测试用；你可以从外部输入
    a = 0
    L = 3.8
    Q_val = 0.0

    E_min =  0.95681763728735669261
    E_max =  0.97603696503974102239

    # n=1000 做粗采样，你可以根据情况调大一点以得到更精细的 bracket
    results_data = main(a, L, Q_val, E_min, E_max, n=1000, tol=1e-7)